<a href="https://colab.research.google.com/github/furk4nkasap/Cardan-optimization-tool/blob/main/CardanJoint_Optizimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Circle, FancyArrowPatch
from ipywidgets import interactive_output, FloatSlider, Dropdown, VBox, Layout, HTML

# -------------------------
# Colors
# -------------------------
ORANGE = "#F28E2B"
GREEN  = "#59A14F"
BLUE   = "#4E79A7"
PURPLE = "#B07AA1"
GRAY   = "#9A9A9A"
BLACK  = "black"
SEG_COLORS = [BLUE, ORANGE, GREEN, PURPLE]

# -------------------------
# Visual parameters
# -------------------------
SHAFT_LEN = 1.35

# β styling (clean)
BETA_ARC_R         = 0.19
BETA_ARC_OFFSET    = 0.20
BETA_ARC_LW        = 3.2
BETA_RAY_LW        = 2.6
BETA_LABEL_FS      = 13
BETA_LABEL_PUSH    = 0.30   # smaller => closer to shaft
THETA0_FS          = 11

# φ styling (end-view)
PHI_R              = 1.30
PHI_ARC_LW         = 2.8
PHI_ARROW_SCALE    = 18
PHI_FS             = 13

# Side-view (shaft + C at end)
C_RING_R           = 1.25
C_RING_LW          = 6.0

# C-ring mouth angles (fixed reference) - defined globally for efficiency
MOUTH_FIXED_HALF_GAP_DEG = 45.0 # (90.0 mouth_deg_fixed / 2)
MOUTH_START_ANGLE_RAD = np.deg2rad(MOUTH_FIXED_HALF_GAP_DEG)
MOUTH_END_ANGLE_RAD = np.deg2rad(360.0 - MOUTH_FIXED_HALF_GAP_DEG)
C_RING_ARC_POINTS = np.linspace(MOUTH_START_ANGLE_RAD, MOUTH_END_ANGLE_RAD, 260)

# ============================================================
# Kinematics  (FIXED)
# ============================================================
def hooke_q(theta_deg, beta_deg):
    """
    Instantaneous speed ratio of a Hooke (Cardan/U-joint):
      q = ω_out / ω_in = cosβ / (1 - sin^2β * cos^2θ)
    """
    t = np.deg2rad(np.asarray(theta_deg, dtype=float))
    b = np.deg2rad(float(beta_deg))
    return np.cos(b) / (1.0 - (np.sin(b)**2) * (np.cos(t)**2))

def _hooke_theta_out(theta_in_rad, beta_rad):
    """
    Hooke position relation (principal branch):
      tan(θ_out) = tan(θ_in) * cosβ
    """
    return np.arctan(np.tan(theta_in_rad) * np.cos(beta_rad))

def single_cardan_q(theta1_deg, beta1_deg=25.0, theta0_deg=0.0):
    return hooke_q(np.asarray(theta1_deg, dtype=float) + float(theta0_deg), float(beta1_deg))

def double_cardan_q(theta1_deg, beta1_deg=25.0, beta2_deg=25.0, phi1_deg=0.0, theta0_deg=0.0):
    """
    Double Cardan (2 joints) – correct intermediate propagation:
      Joint-1: θ1 -> θ2 via Hooke relation
      Phase:   θ2' = θ2 - φ1
      Joint-2: θ2' -> output
    Total q = q1(θ1,β1) * q2(θ2',β2)
    """
    theta1_deg = np.asarray(theta1_deg, dtype=float)

    t1 = np.deg2rad(theta1_deg + float(theta0_deg))
    b1 = np.deg2rad(float(beta1_deg))
    b2 = np.deg2rad(float(beta2_deg))
    phi1 = np.deg2rad(float(phi1_deg))

    # intermediate shaft angle
    t2 = _hooke_theta_out(t1, b1)
    t2p = t2 - phi1

    q1 = np.cos(b1) / (1.0 - (np.sin(b1)**2) * (np.cos(t1)**2))
    q2 = np.cos(b2) / (1.0 - (np.sin(b2)**2) * (np.cos(t2p)**2))
    return q1 * q2

def triple_cardan_q(theta1_deg, beta1_deg=25.0, beta2_deg=25.0, beta3_deg=25.0,
                    phi1_deg=0.0, phi2_deg=0.0, theta0_deg=0.0):
    """
    Triple Cardan (3 joints) – correct propagation:
      Joint-1: θ1 -> θ2
      Phase:   θ2' = θ2 - φ1
      Joint-2: θ2' -> θ3
      Phase:   θ3' = θ3 - φ2
      Joint-3: θ3' -> output
    Total q = q1(θ1,β1) * q2(θ2',β2) * q3(θ3',β3)
    """
    theta1_deg = np.asarray(theta1_deg, dtype=float)

    t1 = np.deg2rad(theta1_deg + float(theta0_deg))
    b1 = np.deg2rad(float(beta1_deg))
    b2 = np.deg2rad(float(beta2_deg))
    b3 = np.deg2rad(float(beta3_deg))
    phi1 = np.deg2rad(float(phi1_deg))
    phi2 = np.deg2rad(float(phi2_deg))

    t2 = _hooke_theta_out(t1, b1)
    t2p = t2 - phi1

    t3 = _hooke_theta_out(t2p, b2)
    t3p = t3 - phi2

    q1 = np.cos(b1) / (1.0 - (np.sin(b1)**2) * (np.cos(t1)**2))
    q2 = np.cos(b2) / (1.0 - (np.sin(b2)**2) * (np.cos(t2p)**2))
    q3 = np.cos(b3) / (1.0 - (np.sin(b3)**2) * (np.cos(t3p)**2))
    return q1 * q2 * q3

def unevenness_percent(q):
    q = np.asarray(q, dtype=float)
    qbar = float(np.mean(q))
    dq = float(np.max(q) - np.min(q))
    return 100.0 * dq / max(qbar, 1e-12)

def optimize_phase(mode, theta_grid_deg, beta1_deg, beta2_deg, beta3_deg, theta0_deg, phi_step_opt):
    """
    Optimization: brute-force scan of phase angles to minimize Δq/q̄ (%).
    Note: Ripple metric is typically 180°-periodic in yoke phasing; however,
          to match your current UI, we scan 0..360.
    """
    theta_grid_deg = np.asarray(theta_grid_deg, dtype=float)
    step = float(phi_step_opt)

    if mode == 1:
        q = single_cardan_q(theta_grid_deg, beta1_deg=float(beta1_deg), theta0_deg=float(theta0_deg))
        return {"phi1": None, "phi2": None, "q_best": q, "unev_best": unevenness_percent(q)}

    phi_vals = np.arange(0.0, 360.0 + 1e-9, step)

    if mode == 2:
        best = None
        for phi1 in phi_vals:
            q = double_cardan_q(theta_grid_deg,
                                beta1_deg=float(beta1_deg),
                                beta2_deg=float(beta2_deg),
                                phi1_deg=float(phi1),
                                theta0_deg=float(theta0_deg))
            u = unevenness_percent(q)
            if (best is None) or (u < best["unev_best"]):
                best = {"phi1": float(phi1), "phi2": None, "q_best": q, "unev_best": u}
        return best

    # mode == 3
    best = None
    for phi1 in phi_vals:
        for phi2 in phi_vals:
            q = triple_cardan_q(theta_grid_deg,
                                beta1_deg=float(beta1_deg),
                                beta2_deg=float(beta2_deg),
                                beta3_deg=float(beta3_deg),
                                phi1_deg=float(phi1),
                                phi2_deg=float(phi2),
                                theta0_deg=float(theta0_deg))
            u = unevenness_percent(q)
            if (best is None) or (u < best["unev_best"]):
                best = {"phi1": float(phi1), "phi2": float(phi2), "q_best": q, "unev_best": u}
    return best

# ============================================================
# (A) Figure-A: q_total plot
# ============================================================
def plot_q_total(mode, beta1_deg, beta2_deg, beta3_deg, phi1_deg, phi2_deg, theta0_deg, phi_step_opt):
    theta = np.linspace(0, 360, 721)

    if mode == 1:
        q_cur = single_cardan_q(theta, beta1_deg=beta1_deg, theta0_deg=theta0_deg)
    elif mode == 2:
        q_cur = double_cardan_q(theta, beta1_deg=beta1_deg, beta2_deg=beta2_deg,
                                phi1_deg=phi1_deg, theta0_deg=theta0_deg)
    else:
        q_cur = triple_cardan_q(theta, beta1_deg=beta1_deg, beta2_deg=beta2_deg, beta3_deg=beta3_deg,
                                phi1_deg=phi1_deg, phi2_deg=phi2_deg, theta0_deg=theta0_deg)

    unev_cur = unevenness_percent(q_cur)

    best = optimize_phase(mode, theta, beta1_deg, beta2_deg, beta3_deg, theta0_deg, phi_step_opt)
    q_opt = best["q_best"]
    unev_opt = best["unev_best"]

    fig, ax = plt.subplots(figsize=(11.5, 4.2))
    ax.plot(theta, q_cur, lw=2.6, label="Current")
    ax.plot(theta, q_opt, lw=2.6, linestyle="--", label="Optimized")

    ax.set_xlabel("Input shaft rotation angle (deg)")
    ax.set_ylabel(r"$q_{total}=\omega_{out}/\omega_{in}$")
    ax.set_title("Figure-A: Unevenness (q_total)")
    ax.grid(True, alpha=0.25)
    ax.legend(loc="best")

    status = "OK" if unev_cur <= 5.0 else "Warning"
    txt = (f"Current Δq/q̄ = {unev_cur:.2f}%  → {status}\n"
           f"Optimized Δq/q̄ = {unev_opt:.2f}%\n")
    if mode == 2:
        txt += f"Optimized φ₁ = {best['phi1']:.0f}°"
    elif mode == 3:
        txt += f"Optimized φ₁ = {best['phi1']:.0f}°,  φ₂ = {best['phi2']:.0f}°"

    ax.text(0.02, 0.98, txt, transform=ax.transAxes,
            ha="left", va="top",
            fontsize=11,
            bbox=dict(boxstyle="round,pad=0.35", fc="white", ec="0.35", alpha=0.95))

    plt.show()

# ============================================================
# (B) Figure-B: 2D geometry (colored shafts) + clean β arcs
# ============================================================
def _rot2d(v, ang_deg):
    a = np.deg2rad(ang_deg)
    c, s = np.cos(a), np.sin(a)
    x, y = v
    return np.array([c*x - s*y, s*x + c*y])

def draw_beta_angle(ax, p0, v_ref, v_tgt, label, color=BLACK):
    v_ref = np.array(v_ref, dtype=float)
    v_tgt = np.array(v_tgt, dtype=float)
    v_ref = v_ref / (np.linalg.norm(v_ref) + 1e-12)
    v_tgt = v_tgt / (np.linalg.norm(v_tgt) + 1e-12)

    ang1 = np.degrees(np.arctan2(v_ref[1], v_ref[0]))
    ang2 = np.degrees(np.arctan2(v_tgt[1], v_tgt[0]))
    d = (ang2 - ang1 + 180) % 360 - 180

    p0 = np.array(p0, dtype=float)
    r = BETA_ARC_R
    offset = BETA_ARC_OFFSET

    ray_len = r + offset + 0.15
    p1 = p0 + ray_len * v_ref
    p2 = p0 + ray_len * v_tgt
    ax.plot([p0[0], p1[0]], [p0[1], p1[1]], lw=BETA_RAY_LW, color=color, solid_capstyle="round")
    ax.plot([p0[0], p2[0]], [p0[1], p2[1]], lw=BETA_RAY_LW, color=color, solid_capstyle="round")

    arc_r = r + offset
    arc = Arc((p0[0], p0[1]), 2*arc_r, 2*arc_r, angle=0,
              theta1=ang1, theta2=ang1 + d, lw=BETA_ARC_LW, color=color)
    ax.add_patch(arc)

    amid = np.deg2rad(ang1 + 0.5*d)
    push = arc_r + BETA_LABEL_PUSH
    tx = p0[0] + push*np.cos(amid)
    ty = p0[1] + push*np.sin(amid)
    ax.text(tx, ty, label, fontsize=BETA_LABEL_FS, color=color, ha="center", va="center")

def draw_geometry_2d(mode, beta1_deg, beta2_deg, beta3_deg, theta0_deg):
    fig, ax = plt.subplots(figsize=(10.8, 4.4))
    ax.set_aspect("equal")
    ax.axis("off")
    ax.set_title("Figure-B: 2D Geometry – β display", fontsize=13)

    # ---- Shaft-1 (input) ----
    p0 = np.array([0.0, 0.0])
    v0 = _rot2d([1.0, 0.0], theta0_deg)
    p1 = p0 + SHAFT_LEN * v0
    ax.plot([p0[0], p1[0]], [p0[1], p1[1]],
            lw=7.0, color=SEG_COLORS[0], solid_capstyle="round")

    # ---- Shaft-2 direction always defined from β1 (for mode>=1) ----
    if mode >= 1:
        v1 = _rot2d(v0, beta1_deg)
        p2 = p1 + SHAFT_LEN * v1

        # 1-Cardan MUST show the output shaft + β1
        if mode == 1:
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]],
                    lw=7.0, color=SEG_COLORS[1], solid_capstyle="round")
            draw_beta_angle(ax, p1, v0, v1, "β₁", color=BLACK)

    # ---- 2-Cardan: draw Shaft-2 + β1, then Shaft-3 + β2 ----
    if mode >= 2:
        # draw middle shaft (Shaft-2) and β1
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]],
                lw=7.0, color=SEG_COLORS[1], solid_capstyle="round")
        draw_beta_angle(ax, p1, v0, v1, "β₁", color=BLACK)

        v2 = _rot2d(v1, beta2_deg)
        p3 = p2 + SHAFT_LEN * v2

        if mode == 2:
            # draw output shaft (Shaft-3) and β2
            ax.plot([p2[0], p3[0]], [p2[1], p3[1]],
                    lw=7.0, color=SEG_COLORS[2], solid_capstyle="round")
            draw_beta_angle(ax, p2, v1, v2, "β₂", color=BLACK)

    # ---- 3-Cardan: keep existing 4-shaft behavior (β1, β2, β3) ----
    if mode >= 3:
        # draw Shaft-3 and β2 (not only for mode==2)
        ax.plot([p2[0], p3[0]], [p2[1], p3[1]],
                lw=7.0, color=SEG_COLORS[2], solid_capstyle="round")
        draw_beta_angle(ax, p2, v1, v2, "β₂", color=BLACK)

        v3 = _rot2d(v2, beta3_deg)
        p4 = p3 + SHAFT_LEN * v3
        ax.plot([p3[0], p4[0]], [p3[1], p4[1]],
                lw=7.0, color=SEG_COLORS[3], solid_capstyle="round")
        draw_beta_angle(ax, p3, v2, v3, "β₃", color=BLACK)

    ax.text(0.0, -0.70, f"θ₀ = {theta0_deg:.0f}°",
            fontsize=THETA0_FS, color=BLACK, ha="left", va="top")
    plt.show()


# ============================================================
# (C) Phase: End-view + Side-view
# ============================================================
def draw_end_view(ax, phi_deg, title, color):
    ax.set_aspect("equal")
    ax.axis("off")

    phi = float(phi_deg) % 360.0

    circ = Circle((0, 0), PHI_R, fill=False, lw=2.2, ec=color)
    ax.add_patch(circ)

    ax.plot([-PHI_R, PHI_R], [0, 0], lw=2.0, color=color)

    a0 = 0.0
    a1 = phi
    theta1 = a0
    theta2 = a1
    arc = Arc((0, 0), 2*PHI_R*0.95, 2*PHI_R*0.95, angle=0,
              theta1=min(theta1, theta2), theta2=max(theta1, theta2),
              lw=PHI_ARC_LW, color=color)
    ax.add_patch(arc)

    ang_rad = np.deg2rad(phi)
    ax.plot([0, PHI_R*np.cos(ang_rad)], [0, PHI_R*np.sin(ang_rad)], lw=2.0, color=color)

    cwccw = "CCW" if (phi % 360.0) <= 180 else "CW"
    ax.text(0.0, 1.85, f"{title}:  φ = {phi:.0f}° ({cwccw})",
            fontsize=12, ha="left", va="bottom", color=BLACK)

    ax.set_xlim(-2.2, 2.2)
    ax.set_ylim(-2.2, 2.2)

def draw_side_view_c_yoke(ax, phi_deg, title,
                         left_c_color=GRAY, shaft_color=BLACK, right_c_color=GRAY):
    ax.set_aspect("equal")
    ax.axis("off")

    phi = float(phi_deg) % 360.0

    depth_factor = abs(np.sin(np.deg2rad(phi)))
    tilt_deg = 90.0 * depth_factor
    b_scale = max(np.cos(np.deg2rad(tilt_deg)), 0.02)

    shaft_len = 4.0
    ax.plot([0.0, shaft_len], [0.0, 0.0],
            lw=6.0, color=shaft_color, solid_capstyle="round")

    t = C_RING_ARC_POINTS

    cxL = -C_RING_R * 0.85
    cy = 0.0
    xL = cxL + C_RING_R * (-np.cos(t))
    yL = cy + C_RING_R * (1.0) * np.sin(t)
    ax.plot(xL, yL, lw=C_RING_LW, color=left_c_color, solid_capstyle="round")

    cxR = shaft_len + C_RING_R * 0.85
    xR = cxR + C_RING_R * np.cos(t)
    yR = cy + (C_RING_R * b_scale) * np.sin(t)
    ax.plot(xR, yR, lw=C_RING_LW, color=right_c_color, solid_capstyle="round")

    ax.text(0.0, 1.90, f"{title} (side view)", fontsize=12, ha="left", va="bottom", color=BLACK)
    ax.text(0.0, 1.55, f"φ = {phi:.0f}°", fontsize=11, ha="left", va="bottom", color=BLACK)

    ax.set_xlim(cxL - C_RING_R - 0.6, cxR + C_RING_R + 1.4)
    ax.set_ylim(-2.2, 2.2)

def draw_phase_figure(mode, phi1_deg, phi2_deg):
    if mode < 2:
        return

    title_12 = "Joint-1 → Joint-2"
    title_23 = "Joint-2 → Joint-3"

    rows = 1 if mode == 2 else 2
    fig = plt.figure(figsize=(14.5, 4.8*rows))
    gs = fig.add_gridspec(rows, 2, width_ratios=[1.15, 0.85], height_ratios=[1]*rows)

    ax_end1  = fig.add_subplot(gs[0, 0])
    ax_side1 = fig.add_subplot(gs[0, 1])

    draw_end_view(ax_end1, phi1_deg, title_12, ORANGE)
    draw_side_view_c_yoke(
        ax_side1, phi1_deg, title_12,
        left_c_color=BLUE, shaft_color=ORANGE, right_c_color=GREEN
    )

    posL = ax_end1.get_position()
    posR = ax_side1.get_position()

    xL = 0.5 * (posL.x0 + posL.x1)
    xR = 0.5 * (posR.x0 + posR.x1)
    yH = max(posL.y1, posR.y1) + 0.015

    fig.text(xL, yH, "END VIEW",
             ha="center", va="bottom",
             fontsize=15, fontweight="bold", color=BLACK)
    fig.text(xR, yH, "SIDE VIEW",
             ha="center", va="bottom",
             fontsize=15, fontweight="bold", color=BLACK)

    if mode == 3:
        ax_end2  = fig.add_subplot(gs[1, 0])
        ax_side2 = fig.add_subplot(gs[1, 1])

        draw_end_view(ax_end2, phi2_deg, title_23, GREEN)
        draw_side_view_c_yoke(
            ax_side2, phi2_deg, title_23,
            left_c_color=ORANGE, shaft_color=GREEN, right_c_color=PURPLE
        )

    fig.subplots_adjust(top=0.86)
    fig.suptitle("C) Phase (φ)", fontsize=18, fontweight="bold", y=0.97)

    plt.show()

# ============================================================
# (D) Extra schematic
# ============================================================
def draw_extra_schematic_single_row_grayscale():
    fig, ax = plt.subplots(figsize=(12.8, 6.2))
    ax.set_aspect("equal")
    ax.axis("off")

    y_top = +2.0
    y_bot = -2.0
    shaft_x0, shaft_x1 = 0.0, 4.2

    ax.text(0.0, 4.2, "Extra schematic – phase interpretation (grayscale)", fontsize=14,
            ha="left", color=BLACK)

    def _row(y, scale_y, label_text):
        ax.plot([shaft_x0, shaft_x1], [y, y], lw=6.0, color=BLACK, solid_capstyle="round")

        t = C_RING_ARC_POINTS

        cxL = -C_RING_R * 0.85
        xL = cxL + C_RING_R * (-np.cos(t))
        yL = y + (C_RING_R * 1.0) * np.sin(t)
        ax.plot(xL, yL, lw=C_RING_LW, color=BLACK, solid_capstyle="round")

        cxR = shaft_x1 + C_RING_R * 0.85
        xR = cxR + C_RING_R * np.cos(t)
        yR = y + (C_RING_R * scale_y) * np.sin(t)
        ax.plot(xR, yR, lw=C_RING_LW, color=BLACK, solid_capstyle="round")

        ax.text(shaft_x0, y + 1.45, label_text, fontsize=12, ha="left", va="bottom", color=BLACK)

    _row(y_top, 1.00, "φ = 0° / 180° / 360°  → face-on (circle-like)")
    _row(y_bot, 0.02, "φ = 90° / 270°  → edge-on (line-like)")

    ax.set_xlim(-3.0, 7.8)
    ax.set_ylim(-3.4, 5.0)
    plt.show()

# ============================================================
# Orchestrator
# ============================================================
def run_all(mode, beta1_deg, beta2_deg, beta3_deg, phi1_deg, phi2_deg, theta0_deg, phi_step_opt):
    plot_q_total(mode, beta1_deg, beta2_deg, beta3_deg, phi1_deg, phi2_deg, theta0_deg, phi_step_opt)
    draw_geometry_2d(mode, beta1_deg, beta2_deg, beta3_deg, theta0_deg)

    if mode >= 2:
        draw_phase_figure(mode, phi1_deg, phi2_deg)
        draw_extra_schematic_single_row_grayscale()

# ============================================================
# Guide text
# ============================================================
guide_tr = """
<b>KULLANIM KILAVUZU (TR)</b><br><br>

<b>Genel</b><br>
Bu araç, kardan (Cardan / U-joint) sistemlerinde <b>açısal hız oranı dalgalanmasını</b>
(unevenness / q_total ripple) analiz eder ve bu dalgalanmayı <b>faz (clocking) açıları</b>
optimizasyonu ile minimize etmeyi hedefler. Tanımlanan mafsal geometrisine bağlı olarak
hız düzgünsüzlüğü hesaplanır ve uygun faz kombinasyonları otomatik olarak belirlenir.<br><br>

<b>1) System</b><br>
Analiz edilecek konfigürasyonu seçiniz.<br>
• <b>1 Cardan</b>: Tek mafsal (φ yok).<br>
• <b>2 Cardan</b>: Çift mafsal (φ₁ aktif).<br>
• <b>3 Cardan</b>: Üç mafsal (φ₁ ve φ₂ aktif).<br><br>

<b>2) Parametre Tanımları</b><br>
<b>β₁, β₂, β₃</b>: Şaft <b>hizasızlık (misalignment)</b> açılarıdır. Hangi şaft çifti arasında
olduğu Figure-B’de gösterilir.<br>
<b>φ₁, φ₂</b>: Komşu mafsal çatallarının <b>şaft ekseni etrafındaki</b> göreli dönüklüğünü ifade eden
<b>faz (clocking)</b> açılarıdır (Figure-C).<br>
<b>θ₀</b>: 2D şemada ilk şaftın başlangıç yönelimidir; hesaplamayı etkilemez, sadece görsel
yerleşimi döndürür (Figure-B).<br>
<b>opt step</b>: Faz tarama adımı (°). Küçük adım → daha iyi optimum, daha uzun hesaplama.<br><br>

<b>Figure-A: Unevenness (q_total)</b><br>
• q_total = ω_out / ω_in, 0–360° boyunca çizilir.<br>
• <b>Current</b>: Mevcut (β, φ, θ₀) ayarları.<br>
• <b>Optimized</b>: Seçilen opt step ile faz taraması sonucu bulunan minimum ripple.<br>
• Bilgi kutusu <b>Δq/q̄ (%)</b> değerini verir. Literatür referansına göre <b>q_total ≤ 5%</b> kinematik olarak kabul edilebilir aralık olarak alınır.<br><br>

<b>Figure-B: 2D Geometry – β Display</b><br>
• Şaftlar renkli segmentlerdir.<br>
• β açıları yalnızca <b>iki ışın + bir yay + etiket</b> ile gösterilir (ok/leader/projection yok).<br><br>

<b>Figure-C: Phase (φ) – End-view + Side-view</b><br>
<b>End-view</b>: φ, referans çap çizgisine göre ölçülür (CW/CCW gösterilir).<br>
<b>Side-view</b>: Solda sabit <b>REF</b> ters-C; sağda φ’ya bağlı şekil değiştiren <b>MOVING</b> C bulunur.
MOVING C görünümü φ arttıkça <b>daire → elips → çizgi</b> davranışı gösterir; bu, faz referansını netleştirir.<br><br>

<b>Extra schematic</b><br>
Bu bölüm renksizdir ve fazın sezgisel yorumunu verir:
φ=0°/180°/360° → face-on (daire gibi), φ=90°/270° → edge-on (çizgi gibi).<br>
"""

guide_en = """
<hr><b>USER GUIDE (EN)</b><br><br>

<b>Overview</b><br>
This tool analyzes angular velocity ratio ripple (<b>q_total</b>) in Cardan (U-joint)
systems and visualizes how <b>phase (clocking) angles</b> can be used to reduce
that ripple. Based on the defined joint geometry, the system evaluates velocity
unevenness and identifies optimal phase combinations.<br><br>

<b>1) System</b><br>
• <b>1 Cardan</b>: Single joint configuration (no phase angle).<br>
• <b>2 Cardan</b>: Double joint configuration (φ₁ enabled).<br>
• <b>3 Cardan</b>: Triple joint configuration (φ₁ and φ₂ enabled).<br><br>

<b>2) Parameter Definitions</b><br>
<b>β₁, β₂, β₃</b>: Shaft <b>misalignment angles</b>. The specific shaft pairs are indicated in Figure-B.<br>
<b>φ₁, φ₂</b>: <b>Phase (clocking) angles</b>, i.e., the relative rotation of adjacent yokes about the <b>shaft axis</b> (Figure-C).<br>
<b>θ₀</b>: Initial orientation of the first shaft in the 2D schematic (rotates the entire layout in Figure-B).<br>
<b>opt step</b>: Optimization scan step (degrees). Smaller steps yield higher accuracy at the cost of longer computation time.<br><br>

<b>Figure-A: Unevenness (q_total)</b><br>
• q_total = ω_out / ω_in plotted over 0–360°.<br>
• <b>Current</b>: Current configuration (β, φ, θ₀).<br>
• <b>Optimized</b>: Minimum ripple found by scanning φ with the selected opt step.<br>
• The info box reports <b>Δq/q̄ (%)</b>. According to academic references, <b>q_total ≤ 5%</b> is considered kinematically acceptable.<br><br>

<b>Figure-B: 2D Geometry – β Display</b><br>
• Shafts are displayed as colored segments.<br>
• Each β angle is drawn using <b>two rays + an arc + a label only</b> (no arrows, no leaders, no projections).<br><br>

<b>Figure-C: Phase (φ) – End-view + Side-view</b><br>
<b>End-view</b>: φ is measured from a reference diameter line; CW / CCW is indicated.<br>
<b>Side-view</b>: The left side is a fixed mirrored-C (<b>REF</b>); the right side is the <b>MOVING</b> C.
The MOVING C transitions <b>circle → ellipse → line</b> as φ approaches edge-on, making the phase reference explicit.<br><br>

<b>Extra schematic</b><br>
Fully grayscale intuitive phase view:
φ=0°/180°/360° → face-on (circle-like), φ=90°/270° → edge-on (line-like).<br>
"""

help_txt = HTML(guide_tr + guide_en)

# ============================================================
# UI
# ============================================================
mode_dd = Dropdown(
    options=[("1 Cardan (single)", 1), ("2 Cardan (double)", 2), ("3 Cardan (triple)", 3)],
    value=3,
    description="System:",
    layout=Layout(width="320px")
)

beta1_sl = FloatSlider(value=25, min=0, max=60, step=1, description="β₁ (deg)")
beta2_sl = FloatSlider(value=25, min=0, max=60, step=1, description="β₂ (deg)")
beta3_sl = FloatSlider(value=25, min=0, max=60, step=1, description="β₃ (deg)")

phi1_sl  = FloatSlider(value=0,  min=0, max=360, step=1, description="φ₁ (deg)")
phi2_sl  = FloatSlider(value=0,  min=0, max=360, step=1, description="φ₂ (deg)")

theta0_sl = FloatSlider(value=0, min=0, max=180, step=1, description="θ₀ (deg)")
phi_step_sl = FloatSlider(value=5, min=1, max=10, step=1, description="opt step")

def ui_visibility(mode):
    beta2_sl.layout.display = "none" if mode == 1 else "flex"
    beta3_sl.layout.display = "none" if mode != 3 else "flex"
    phi1_sl.layout.display  = "none" if mode == 1 else "flex"
    phi2_sl.layout.display  = "none" if mode != 3 else "flex"

ui_out = interactive_output(ui_visibility, {"mode": mode_dd})

main_out = interactive_output(
    run_all,
    {
        "mode": mode_dd,
        "beta1_deg": beta1_sl,
        "beta2_deg": beta2_sl,
        "beta3_deg": beta3_sl,
        "phi1_deg": phi1_sl,
        "phi2_deg": phi2_sl,
        "theta0_deg": theta0_sl,
        "phi_step_opt": phi_step_sl
    }
)

VBox([
    mode_dd,
    ui_out,
    beta1_sl, beta2_sl, beta3_sl,
    phi1_sl, phi2_sl,
    theta0_sl,
    phi_step_sl,
    main_out,
    help_txt
])
